# Experiment with extracting shape info from confocal images of fish embryos

In [ ]:
from aicsimageio import AICSImage
import os 

# define save paths
image_name = "1A_LM010_RT_kikume"
read_path = "/Users/nick/Dropbox (Cole Trapnell's Lab)/Nick/morphMap/data/yx1_samples/20230322/RT/"
image_path = read_path + image_name + ".nd2"
pcd_path = "/Users/nick/Dropbox (Cole Trapnell's Lab)/Nick/morphMap/data/yx1_pcd/20230322/RT/"

# make save directory
if not os.path.isdir(pcd_path):
    os.makedirs(pcd_path)

# load image
imObject = AICSImage(image_path)

In [ ]:
# upsample along z axis
import scipy
import numpy as np

# get resolution
res_raw = imObject.physical_pixel_sizes
res_array = np.asarray(res_raw)
res_array = np.insert(res_array, 0, 1)
pixel_size_z = res_array[1]
pixel_size_x = res_array[2]
pixel_size_y = res_array[3]

# extract raw image
imData = np.squeeze(imObject.data)

z_rs_factor = pixel_size_z/pixel_size_x
print(z_rs_factor)

ds_factor = 2
pixel_size_new = pixel_size_x / ds_factor
imData_rs = scipy.ndimage.zoom(imData, [z_rs_factor/ds_factor, 1/ds_factor, 1/ds_factor])

# Step1: obtain rough "surface prior"
For each xy coordinate in image, determine whether it contains "inside" pixels. Then, find the surface pixel for those that do have one ore more inside pixels.

The most naive approach I can imagine is taking the brightest pixel in each Z column as my surface point. Let's try that first

In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px



# find brightest pixel

max_pos_z = np.argmax(imData_rs, axis=0)
max_brightness_z = np.max(imData_rs, axis=0)

# generate x and y axes
xg, yg = np.meshgrid(range(max_pos_z.shape[1]), range(max_pos_z.shape[0]))

im95 = np.percentile(max_brightness_z, 90)
x_plot = xg[np.where(max_brightness_z>=im95)]
y_plot = yg[np.where(max_brightness_z>=im95)]
z_plot = max_pos_z[np.where(max_brightness_z>=im95)]

fig = px.scatter_3d(x=x_plot*pixel_size_new, 
                    y=y_plot*pixel_size_new,
                    z=z_plot*pixel_size_new,
                    opacity=0.02,
                    color=z_plot)
fig.show()

**Try to convert this to a 3D mesh.**

In [ ]:
import pyvista as pv
import pymeshfix as mf
from pymeshfix import MeshFix
from pymeshfix._meshfix import PyTMesh
# from open3d.j_visualizer import JVisualizer
import open3d as o3d

# await pv.set_jupyter_backend('trame')
np.random.seed(124)
n_samples = x_plot.size
#index_vec = 5000
mesh_indices = range(0, x_plot.size)#index_vednp.random.choice(index_vec, n_samples)

# convert xyz coordinates to a point cloud object
xyz_array = np.concatenate((np.reshape(x_plot[mesh_indices]*pixel_size_x, (n_samples, 1)),
                            np.reshape(y_plot[mesh_indices]*pixel_size_y, (n_samples, 1)),
                            np.reshape(z_plot[mesh_indices]*pixel_size_x, (n_samples, 1))), axis=1)


# Pass xyz to Open3D.o3d.geometry.PointCloud and visualize
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz_array)
o3d.io.write_point_cloud(pcd_path + image_name + "_full.ply", pcd)

# downsample to a managable number of points
pcd_ds = pcd.voxel_down_sample(voxel_size=2)

# plot point cloud
print(np.asarray(pcd_ds.points).shape)

pv_cloud = pv.PolyData(np.asarray(pcd_ds.points))
pv_cloud.plot(jupyter_backend='ipygany')

**Step 1:** Remove outlier points

In [ ]:
# cl, ind = pcd_ds.remove_statistical_outlier(nb_neighbors=10, std_ratio=3)
# xyz_ds = np.asarray(pcd_ds.points)

# xyz_ds_out = np.delete(xyz_ds, ind, axis=0)
# xyz_ds_in = xyz_ds[ind]

# pcd_in = o3d.geometry.PointCloud()
# pcd_in.points = o3d.utility.Vector3dVector(xyz_ds_in)

# pcd_out = o3d.geometry.PointCloud()
# pcd_out.points = o3d.utility.Vector3dVector(xyz_ds_out)

# pv_cloud_out = pv.PolyData(xyz_ds_out)
# pv_cloud_in = pv.PolyData(xyz_ds_in)
# pv_cloud_in = pcd_ds#.plot(jupyter_backend='ipygany')

**Attempt rough meshing with "raw" points.** Note that this only works reasonanbly when the alpha parameter is set to be larger than the z resoultion in Z (20um in this case)

In [ ]:
# Attempt with delaunay 3d
alpha = 10
dl_3d_mesh = pv_cloud.delaunay_3d(alpha=alpha)
# dl_3d_mesh_alt = pv_cloud.reconstruct_surface()
dl_3d_mesh.plot(show_edges=True, jupyter_backend='ipygany')


In [ ]:
faces = []
i, offset = 0, 0
cc = dl_3d_mesh.cells # fetch up front
while i < dl_3d_mesh.n_cells:
    nn = cc[offset]
    faces.append(cc[offset+1:offset+1+nn])
    offset += nn + 1
    i += 1


In [ ]:
edges = dl_3d_mesh.extract_all_edges()

shell = dl_3d_mesh.extract_geometry()
print(shell.faces)
print(shell.verts)

pv_mesh_name = pcd_path + image_name + "_pv_mesh.ply"
shell.save(pv_mesh_name)

tin = _meshfix.PyTMesh()
tin.load_file(pv_mesh_name)
#shell.plot(show_edges=False, jupyter_backend='ipygany')

In [ ]:
from pymeshfix import _meshfix

fix_size = 200
tin.fill_small_boundaries(nbe=fix_size, refine=True)

# convert to mesh
vert, faces = sf.return_arrays()
triangles = np.empty((faces.shape[0], 4), dtype=faces.dtype)
triangles[:, -3:] = faces
triangles[:, 0] = 3

surf_fix = pv.PolyData(vert, triangles)

surf_fix.plot(show_edges=False, jupyter_backend='panel')

In [ ]:
dl_3d_mesh.__dict__.keys()

This captures gross morphological features, but we give up a lot of xy resolution. How could we do better?

**Attempt 1:** fit spline along z direction

In [ ]:
import scipy

test = scipy.interpolate.bisplrep(xyz_ds_in[:, 0], xyz_ds_in[:, 1], xyz_ds_in[:, 2])

In [ ]:
npoints = 50
xg = np.linspace(np.min(xyz_ds_in[:, 0]), np.max(xyz_ds_in[:, 0]), npoints)
yg = np.linspace(np.min(xyz_ds_in[:, 1]), np.max(xyz_ds_in[:, 1]), npoints)

x_grid, y_grid = np.meshgrid(xg, yg)
test_out = scipy.interpolate.bisplev(xg, yg, test, dx=0, dy=0)


xyz_interp = np.concatenate((np.reshape(x_grid, (x_grid.size, 1)), 
                             np.reshape(y_grid, (x_grid.size, 1)),
                             np.reshape(test_out, (x_grid.size, 1))),
                             axis=1)

xyz_interp_filt = xyz_interp[np.where(xyz_interp[:, 2]<=350)]
xyz_interp_filt = xyz_interp_filt[xyz_interp_filt[:, 2]>=0]
# pcd_interp = o3d.geometry.PointCloud()
# pcd_interp.points = o3d.utility.Vector3dVector(xyz_interp)
# bbox = pcd_ds.get_axis_aligned_bounding_box()
# pcd_interp_crop = pcd_interp.crop(bbox)

pv_cloud_interp = pv.PolyData(xyz_interp_filt)
pv_cloud_interp.plot(jupyter_backend='ipygany')

**Attempt 2:** downsample, interpolate, and then infer mesh

In [ ]:
import numpy as np
import scipy
from scipy import interpolate


f = interpolate.interp2d(xyz_ds_in[:, 0], xyz_ds_in[:, 1], xyz_ds_in[:, 2], kind='linear')


In [ ]:
npoints = 150
xg = np.linspace(np.min(xyz_ds_in[:, 0]), np.max(xyz_ds_in[:, 0]), npoints)
yg = np.linspace(np.min(xyz_ds_in[:, 1]), np.max(xyz_ds_in[:, 1]), npoints)

x_grid, y_grid = np.meshgrid(xg, yg)

znew = f(xg, yg)

xyz_interp = np.concatenate((np.reshape(x_grid, (x_grid.size, 1)), 
                             np.reshape(y_grid, (x_grid.size, 1)),
                             np.reshape(znew, (x_grid.size, 1))),
                             axis=1)

xyz_interp_filt = xyz_interp[np.where(xyz_interp[:, 2]<=350)]
xyz_interp_filt = xyz_interp_filt[xyz_interp_filt[:, 2]>0]
pv_cloud_interp = pv.PolyData(xyz_interp_filt)
pv_cloud_interp.plot(jupyter_backend='ipygany')